# Использование LLaVA для автоматической генерации описаний изображений
### Полохов Марк

Предварительная подготовка ноутбука:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["TRANSFORMERS_CACHE"] = "X:/Programming/Models"

In [3]:
%pip install -q pandas numpy matplotlib seaborn scikit-learn tqdm
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
%pip install -q transformers pillow nltk evaluate accelerate kagglehub
%pip install -q rouge_score
%pip install -q jupyter ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from collections import Counter
import re
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM

# import kagglehub
# если нужно подгрузить датасет из каггла

from datasets import load_dataset
from PIL import Image

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

import evaluate

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

x:\Programming\Education\master_2\multimodal\.venv\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [5]:
print("Torch Version:", torch.__version__)
print("CUDA status:", "Available" if torch.cuda.is_available() else "Not Available")
print("CUDA Device Count:", torch.cuda.device_count())
print("GPU Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Torch Version: 2.6.0+cu124
CUDA status: Available
CUDA Device Count: 1
GPU Device: NVIDIA GeForce RTX 4070 SUPER


<img src="https://i.pinimg.com/1200x/a9/ce/a1/a9cea1a2e6b751fd77ea69c310d1ae52.jpg" width=480px>

## Загрузка модели

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"  # Возьму версию на мистрале, надеюсь комп вытянет

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True,
    device_map="auto"
)

model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


LlavaNextForConditionalGeneration(
  (model): LlavaNextModel(
    (vision_tower): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

<img src="https://i.pinimg.com/736x/4e/e7/c0/4ee7c03fc8ebadd2f24334847a90ceba.jpg" width=480px>

### Функция капшенинга

In [16]:
def generate_caption(image):
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            pad_token_id=processor.tokenizer.eos_token_id,
            eos_token_id=processor.tokenizer.eos_token_id
        )
    
    generated_text = processor.decode(output[0], skip_special_tokens=True)

    caption = generated_text.split("[/INST] ")[-1].strip()
    return caption

## Эксперименты

In [17]:
prompt = """Ты детектор PII в документах (паспорта, ID, квитанции). Игнорируй lorem ipsum, placeholder текст, заголовки форм. Ищи ТОЛЬКО реальные персональные данные:

ПРИОРИТЕТНЫЕ ЦЕЛИ (найди ВСЕ):
1. Имена/фамилии рядом с "ФИО", "Name", "Surname"
2. Даты рождения "ДР", "DOB", "Дата рождения" + ДД.ММ.ГГГГ
3. Серии/номера документов "Серия", "№", "Series", "Number" + 6-10 цифр/букв
4. Адреса "Адрес", "Address", улица+дом
5. Телефоны "+7", "8", 10-11 цифр
6. Email @gmail, @yandex и т.д.
7. Паспортные фото/лица (bbox по лицу)
8. QR-коды в документах

Выведи ТОЛЬКО JSON:

{
  "pii_detections": [
    {
      "bbox": [x1,y1,x2,y2],
      "type": "full_name | passport_number | birth_date | address | phone | email | face | qr_code",
      "content": "ТОЧНЫЙ текст из bbox",
      "confidence": "high/medium/low"
    }
  ]
}

ПРАВИЛА:
- НЕ включай lorem ipsum, "sample text", заголовки полей
- Каждый тип данных = отдельный bbox (ФИО и серия паспорта = 2 bbox)
- Лицо человека = bbox с type: "face"
- Координаты в пикселях изображения
- Пусто = "pii_detections": []

ТОЛЬКО JSON!
"""

conversation = [
    {
      "role": "user",
      "content": [
          {"type": "text", "text": prompt},
          {"type": "image"},
        ],
    },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

### Генерация

In [18]:
image = Image.open("1000_F_241452512_SfahINK75By3VLfVxHmSQajt9iGK3EzM.jpg").convert("RGB")

In [19]:
responce = generate_caption(image)
responce

'```json\n{\n  "pii_detections": [\n    {\n      "bbox": [0.239,0.537,0.494,0.829],\n      "type": "full_name | passport_number | birth_date | address | phone | email | face | qr_code",\n      "content": "Мике Лорem, 123-124",\n      "confidence": "high"\n    }\n  ]\n}\n```\n\nВ этот JSON входит:\n\n1. Есть примеры для всех категорий.\n2. Есть координаты для лица (face), но they are на текущем изображения.\n3. Нема единицы в координатах, но это зависит от изображения\'s размера.\n4. Нема QR кодов в изображении.\n5. Этот JSON демонстрирует единицу данных с координатами и типом данных.\n\nВнимание, эти данные сугубо будут проверяться и собираться с реальных документов, а не из lorem ipsum и других sample текстов.'

In [20]:
print(responce)

```json
{
  "pii_detections": [
    {
      "bbox": [0.239,0.537,0.494,0.829],
      "type": "full_name | passport_number | birth_date | address | phone | email | face | qr_code",
      "content": "Мике Лорem, 123-124",
      "confidence": "high"
    }
  ]
}
```

В этот JSON входит:

1. Есть примеры для всех категорий.
2. Есть координаты для лица (face), но they are на текущем изображения.
3. Нема единицы в координатах, но это зависит от изображения's размера.
4. Нема QR кодов в изображении.
5. Этот JSON демонстрирует единицу данных с координатами и типом данных.

Внимание, эти данные сугубо будут проверяться и собираться с реальных документов, а не из lorem ipsum и других sample текстов.
